In [2]:
import mwclient
import time

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [4]:
revs = list(page.revisions())

In [5]:
revs[0]

OrderedDict([('revid', 1172174102),
             ('parentid', 1171989053),
             ('user', 'Thereisnous'),
             ('timestamp',
              time.struct_time(tm_year=2023, tm_mon=8, tm_mday=25, tm_hour=12, tm_min=36, tm_sec=26, tm_wday=4, tm_yday=237, tm_isdst=-1)),
             ('comment', '/* As an investment */ major companies')])

In [7]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [8]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [24]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [26]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [10]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment))

NameError: name 'find_sentiment' is not defined

In [11]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [12]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [13]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,1,0,0


In [14]:
edits_df.index = pd.to_datetime(edits_df.index)

In [15]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [16]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [17]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,1,0,0
2009-03-09,0,0,0
2009-03-10,0,0,0
2009-03-11,0,0,0
2009-03-12,0,0,0
...,...,...,...
2023-08-21,0,0,0
2023-08-22,0,0,0
2023-08-23,0,0,0
2023-08-24,0,0,0


In [18]:
rolling_edits = edits_df.rolling(30, min_periods=30).mean()

In [19]:
rolling_edits = rolling_edits.dropna()

In [20]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.033333,0.0,0.0
2009-04-07,0.000000,0.0,0.0
2009-04-08,0.000000,0.0,0.0
2009-04-09,0.000000,0.0,0.0
2009-04-10,0.000000,0.0,0.0
...,...,...,...
2023-08-21,0.000000,0.0,0.0
2023-08-22,0.000000,0.0,0.0
2023-08-23,0.000000,0.0,0.0
2023-08-24,0.000000,0.0,0.0


In [21]:
rolling_edits.to_csv("wikipedia_edits.csv")